In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import warnings
warnings.filterwarnings('ignore')

import joblib

from ic.evaluation import evaluate
from ic.models.ensemble import Ensemble

# Avaliação (v1)

Aqui estão os resultados obtidos na primeira leva de testes (conforme conversado em 06/06). 

Conversamos de testar inicialmente `Naive Bayes` e `Hoeffding Tree`, porém decidi incluir também `Compliment Naive Bayes`, pois de acordo com a [descrição](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.ComplementNB.html) *"It is particularly suited for imbalanced data sets."* (conjuntos de dados onde um categoria é bem mais "rara" que a(s) outra(s)), o que pareceu se encaixar bem no problema.

Obs: o código para os modelos está em [ic/models](../ic/models), inicialmente foi feito para que pudesse ser treinado pela linha de comando.

---

## Parte 1

Primeiramente, treinamos/testamos cada modelo (`Gaussia Naive Bayes`, `Complement Naive Bayes` e `Hoeffding Tree`) utilizando a as seguintes estratégias:

1. Treinar com os dados de **terça**, testar com os dados de **quarta-sexta**
2. Treinar com os dados de **quinta**, testar com os dados de **sexta**

Também foi utilizada uma Ensemble dos 3 modelos (prevê como "verdadeiro" se qualquer um dos 3 prever como verdadeiro).

Os resultados/métricas para cada um deles para as duas estratégias podem ser vistos abaixo:

In [2]:
print(evaluate(
    joblib.load('../../../trained_models/naive_bayes_tuesday.joblib'),
    '../../../data/Grouped_Wed-Fri.csv'
))

{'total_analyzed': 1852750, 'accuracy': 0.7070711105114019, 'recall': 0.0, 'precision': 0, 'false_alarm_rate': 0.0}


In [3]:
print(evaluate(
    joblib.load('../../../trained_models/naive_bayes_thursday.joblib'),
    '../../../data/Grouped_Friday.csv'
))

{'total_analyzed': 702718, 'accuracy': 0.5890456769287253, 'recall': 0.0, 'precision': 0, 'false_alarm_rate': 0.0}


In [4]:
print(evaluate(
    joblib.load('../../../trained_models/complement_nb_tuesday.joblib'),
    '../../../data/Grouped_Wed-Fri.csv'
))

{'total_analyzed': 1852750, 'accuracy': 0.545428147348536, 'recall': 0.4474594821677317, 'precision': 0.30928910387198827, 'false_alarm_rate': 0.41398491327653036}


In [5]:
print(evaluate(
    joblib.load('../../../trained_models/complement_nb_thursday.joblib'),
    '../../../data/Grouped_Friday.csv'
))

{'total_analyzed': 702718, 'accuracy': 0.5206142435514673, 'recall': 0.11040739650605122, 'precision': 0.2850450579316264, 'false_alarm_rate': 0.19320034884872672}


In [6]:
print(evaluate(
    joblib.load('../../../trained_models/hoeffding_tuesday.joblib'),
    '../../../data/Grouped_Wed-Fri.csv'
))

{'total_analyzed': 1854916, 'accuracy': 0.706827155515398, 'recall': 0.0, 'precision': 0, 'false_alarm_rate': 0.0}


In [7]:
print(evaluate(
    joblib.load('../../../trained_models/hoeffding_thursday.joblib'),
    '../../../data/Grouped_Friday.csv'
))

{'total_analyzed': 703245, 'accuracy': 0.5891574060249273, 'recall': 0.0, 'precision': 0, 'false_alarm_rate': 0.0}


In [8]:
print(evaluate(
    Ensemble([
        '../../../trained_models/naive_bayes_tuesday.joblib',
        '../../../trained_models/complement_nb_tuesday.joblib',
        '../../../trained_models/hoeffding_tuesday.joblib'
    ]),
    '../../../data/Grouped_Wed-Fri.csv'
))

{'total_analyzed': 1854916, 'accuracy': 0.545372944111755, 'recall': 0.4465650749984829, 'precision': 0.30928910387198827, 'false_alarm_rate': 0.41364421613829555}


In [9]:
print(evaluate(
    Ensemble([
        '../../../trained_models/naive_bayes_thursday.joblib',
        '../../../trained_models/complement_nb_thursday.joblib',
        '../../../trained_models/hoeffding_thursday.joblib'
    ]),
    '../../../data/Grouped_Friday.csv'
))

{'total_analyzed': 703245, 'accuracy': 0.5207772540153147, 'recall': 0.11035466196875984, 'precision': 0.2850450579316264, 'false_alarm_rate': 0.19301895627072663}


---

## Parte 2

Notei que os tipos de ataques são mutualmente exclusivos em cada dia (um dia não possui os mesmos tipos de ataques dos outros). Portanto decidi treinar os mesmos modelos agrupando os dados de todos os dias (deixando 70% para treino e 30% para teste).

Os resultados podem ser vistos nas células seguintes:

In [10]:
print(evaluate(
    joblib.load('../../../trained_models/naive_bayes_all.joblib'),
    '../../../data/Split_All_Test.csv'
))

{'total_analyzed': 1130043, 'accuracy': 0.8036773821881115, 'recall': 0.0, 'precision': 0, 'false_alarm_rate': 0.0}


In [11]:
print(evaluate(
    joblib.load('../../../trained_models/complement_nb_all.joblib'),
    '../../../data/Split_All_Test.csv'
))

{'total_analyzed': 1130043, 'accuracy': 0.5613945664014556, 'recall': 0.5355257760769518, 'precision': 0.23231639832149018, 'false_alarm_rate': 0.43228619561985926}


In [12]:
print(evaluate(
    joblib.load('../../../trained_models/hoeffding_all.joblib'),
    '../../../data/Split_All_Test.csv'
))

{'total_analyzed': 1131157, 'accuracy': 0.8884372372712188, 'recall': 0.4405039831945625, 'precision': 0.9818130777405707, 'false_alarm_rate': 0.0019959333133813353}


In [13]:
print(evaluate(
    Ensemble([
        '../../../trained_models/naive_bayes_all.joblib',
        '../../../trained_models/complement_nb_all.joblib',
        '../../../trained_models/hoeffding_all.joblib'
    ]),
    '../../../data/Split_All_Test.csv'
))

{'total_analyzed': 1131223, 'accuracy': 0.5821752209776498, 'recall': 0.6441349652960519, 'precision': 0.26679124638091584, 'false_alarm_rate': 0.4329793577830246}


---

## Parte 3 

Além disso, devido ao fato de que o `Complemente NB` em teoria é multiclasse e lida bem com probabilidades desproporcionais para cada classe, testei treiná-lo com cada classe de ataque (ao invés de binário "ataque/não ataque"). O resultado está abaixo:

In [2]:
from ic.models.complement_nb_multiclass import process_row as process_row_complement_nb
print(evaluate(
    joblib.load('../../trained_models/complement_nb_multiclass_all.joblib'),
    '../../../data/Split_All_Test.csv',
    process_row_complement_nb
))

{'total_analyzed': 1130043, 'accuracy': 0.4976943355252853, 'recall': 0.818037168755888, 'precision': 0.25606497145646795, 'false_alarm_rate': 0.580559134101895}
